# P2FA Alginment

Katrina Kechun Li, 2023.3

The code below is largely from Chenzi Xu's [tutorial](https://chenzixu.rbind.io/resources/1forcedalignment/), but adapted to my own workflow.

All the codes (includes Terminal ones) is written in a way that can be directly run in this Ipython Notebook.

## Prepare sound files

In [1]:
currentlang = "Changsha"

My current working folder is in GitHub/0 PhD_working folder:

1. run `p2fasound.praat` to segment annotated sounds&textgrids to individual files in `/1 individual`

2. Use sox to generate downsampled sound files, saved to `/downsample` folder

In [2]:
soundfolder = "/Users/kechun/Documents/0_PhD_working_folder/" + currentlang + "/1 Individual"
%cd {soundfolder}
!for i in *.wav; do sox "$i" -r 16k -b 16 -c 1 "downsample/${i%.}"; done

/Users/kechun/Documents/0_PhD_working_folder/Changsha/1 Individual


## Prepare the text files

1. Check that all the transciptions have been listed in the latest annotation excel tab `record`. Copy the excel file to `/processtext`, and then run the following code. Some files will be generated in the folder.

Remember to change the excel file name.

In [ ]:
# After checking the 'record' tab in the annotating excel file, we processed it.
# In the sheet, there should be four columns: Speaker, file, sentence will be used to generate 'tokenid', whereas the last column is 'transcription'. 
####### Change file name here #######
filename = "长沙话标注20231104.xlsx"
# filename = "粤语标注230531.xlsx"
# filename = "成都话标注230930.xlsx"
######################################
import pandas as pd
import os
annofolder = "/Users/kechun/Documents/P2FA_" + str(currentlang.lower()) + "/processtext"
fullname = os.path.join(annofolder,filename)
df = pd.read_excel(fullname, sheet_name = "record")
df["text"] = df['annotation'].str.replace('[^\w\s]','')
df.text = df.text.str.join(" ")
df["tokenid"]=df['Speaker']+ df['file']+ df['sentence'] + ".wav"
output = df[['tokenid', 'text']]
output.to_csv(os.path.join(annofolder, "list.txt"), sep = " ", header = False, index = False)

/var/folders/tf/xp1pp0c905qbfj8gl5gm79h80000gn/T/ipykernel_44802/3539112296.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df['annotation'].str.replace('[^\w\s]','')


2. From list.txt to generate individual text files.

In [9]:
annofolder = "/Users/kechun/Documents/P2FA_" + str(currentlang.lower()) + "/processtext"
%cd {annofolder}
!cat list.txt | while read line || [ -n "$line" ]; do echo $line | awk '{$1=""}1'| awk '{$1=$1}1' |awk '{gsub(/"/, "")} 1' > $(cut -d " " -f1 <<< individualtxt/$line).txt; done < list.txt

/Users/kechun/Documents/P2FA_chengdu/processtext


3. Run the command below to check pronunciation:

In [10]:
annofolder = "/Users/kechun/Documents/P2FA_" + str(currentlang.lower()) + "/processtext"
%cd {annofolder}
!bash checkpron.sh

/Users/kechun/Documents/P2FA_chengdu/processtext


4. Check the `missingwords.txt` as well as the `wordphones.txt` to see if all the characters used in the transcriptions have correct phonetic transcriptions. Modify `dictcopy` accordingly.

5. Repeat Step 3 recursively until all issues are clear

**In addition, make sure there is no empty lines in the dict file. Otherwise P2FA will report errors.**

## Run the P2FA
1. Copy the sound files (from `/downsample`) and text files (from `/individualtext`) to `/run`

2. Copy the `dictcopy` to `/run/model`, and rename as `dict`

2. Make sure the Calign2textgrid.py document 'HOMEDIR' variable shows the right python (where `/run` folder is located)

2. Make sure there is a folder `/textgrid`

3. Run the code below in terminal


In [13]:
runfolder = "/Users/kechun/Documents/P2FA_" + str(currentlang.lower()) + "/run/"
%cd {runfolder}
!for i in *.wav; do python Calign2textgrid.py $i $i.txt textgrid/$i.TextGrid; done

/Users/kechun/Documents/P2FA_chengdu/run


## Check the recordings

1. Move the original sound files (not the downsample one) to `/workflow/sound_original`

2. Move the generated textgrid to `workflow/textgrid_original`

3. Now move on to the file `Workbook_02processing.ipynb` to further process the files.